In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [31]:
data = [[ 0.5, None, None, None, None, None, None, None],#newc
        [ 0.5,  0.3, None, None, None, None, None, None],#birm
        [ 1.0,  0.5, None, None, None, None, None, None],#lond
        [ 0.2,  0.2, None, None, None, None, None, None],#exer
        [ 0.6,  0.4, None, None, None, None, None, None],#bris
        [ 0.4,  0.3, None, None, None, None, None, None],#nort
        [ 1.0,  2.0, None,  1.0, None, None,  1.2, None],#c1
        [None, None,  1.5,  0.5,  1.5, None,  0.6,  0.4],#c2
        [ 1.5, None,  0.5,  0.5,  2.0,  0.2,  0.5, None],#c3
        [ 2.0, None,  1.5,  1.0, None,  1.5, None,  0.5],#c4
        [None, None, None,  0.5,  0.5,  0.5,  0.3,  0.6],#c5
        [ 1.0, None,  1.0, None,  1.5,  1.5,  0.8,  0.9]]#c6

CUSTOMERS = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
FACTORIES = ['LIVERPOOL', 'BRIGHTON'] 
DEPOTS    = ['NEWCASTLE', 'BIRMINGHAM', 'LONDON', 'EXERTER', 'BRISTOL', 'NORTHAMPTON']
DEPOTS_NB = ['NEWCASTLE', 'EXERTER', 'BRISTOL', 'NORTHAMPTON']

supply_cost = pd.DataFrame(data, columns=FACTORIES+DEPOTS, index=DEPOTS+CUSTOMERS).fillna(0.0)
poss_supply = {i: [j for j in DEPOTS+CUSTOMERS if supply_cost[i][j] != 0 ] for i in FACTORIES+DEPOTS}
poss_demand = {j: [i for i in FACTORIES+DEPOTS if supply_cost[i][j] != 0 ] for j in DEPOTS+CUSTOMERS}
connections = [(i,j) for i in poss_supply.keys() for j in poss_supply[i]]

depots_costs = dict(zip(['BRISTOL', 'NORTHAMPTON', 'BIRMINGHAM'], [12000, 4000, 3000]))
depots_saves = dict(zip(['NEWCASTLE', 'EXERTER'], [10000, 5000]))

capacity  = dict(zip(FACTORIES + DEPOTS, [150000, 200000, 70000, 50000, 100000, 40000, 30000, 25000]))
requirem  = dict(zip(CUSTOMERS, [50000, 10000, 40000, 35000, 60000, 20000]))

In [88]:
model = gp.Model('12 Depot Location')

# add vars
supplyto = model.addVars(connections,
                         name='supplyto')
active_d = model.addVars(DEPOTS,
                         vtype=GRB.BINARY,
                         name='active_d')

# objective function
model.setObjective((gp.quicksum(supply_cost[i][j]*supplyto[i,j] for i, j in connections) 
                    + gp.quicksum(depots_costs[i]*active_d[i] for i in depots_costs.keys())
                    + gp.quicksum(depots_saves[i]*active_d[i] for i in depots_saves.keys()) - 15000))

# add constraints

model.addConstrs((gp.quicksum(supplyto[i,j] for i in poss_demand[j]) == gp.quicksum(supplyto[j,k] for k in poss_supply[j]) for j in DEPOTS),
                 name='depot_cap')                      # into depot == out of depot
model.addConstrs((gp.quicksum(supplyto[i,j] for i in poss_demand[j]) >= requirem[j] for j in CUSTOMERS),
                 name='requirement')

model.addConstr((active_d['NEWCASTLE']+active_d['EXERTER']+active_d['BRISTOL']+active_d['NORTHAMPTON'] <= 2),
                name='max_n_depots')

model.addConstrs((gp.quicksum(supplyto[i,j] for j in poss_supply[i]) <= capacity[i]*active_d[i] for i in DEPOTS_NB),
                 name='capacity')

model.addConstr((gp.quicksum(supplyto[i,'BIRMINGHAM'] for i in poss_demand['BIRMINGHAM']) <= capacity['BIRMINGHAM'] + 20000*active_d['BIRMINGHAM']),
                 name='capacity[BIRMINGHAM]')

model.update()

In [89]:
model.write('12 Depot Location.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 18 rows, 48 columns and 96 nonzeros
Model fingerprint: 0x30e955db
Variable types: 42 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+04]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 6e+04]
Presolve removed 8 rows and 25 columns
Presolve time: 0.00s
Presolved: 10 rows, 23 columns, 41 nonzeros
Variable types: 18 continuous, 5 integer (5 binary)

Root relaxation: objective 1.740000e+05, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    174000.00000 174000.000  0.00%     -    0s

Explored 0 nodes (12 simplex iterations) in 0.04 seconds
Thread count was 8 (of 8 available processors)

Solution c

In [90]:
print('Distribution cost of ${:.2f}'.format(model.objVal))

dist_solution = pd.DataFrame([], columns=FACTORIES+DEPOTS, index=DEPOTS+CUSTOMERS).fillna(0)
for i, t in supplyto.keys():
    dist_solution[i][t] = supplyto[i,t].x
print('===DISTRIBUTION PLAN')
dist_solution

Distribution cost of $174000.00
===DISTRIBUTION PLAN


,LIVERPOOL,BRIGHTON,NEWCASTLE,BIRMINGHAM,LONDON,EXERTER,BRISTOL,NORTHAMPTON
NEWCASTLE,0,0,0,0,0,0,0,0
BIRMINGHAM,0,70000,0,0,0,0,0,0
LONDON,0,10000,0,0,0,0,0,0
EXERTER,40000,0,0,0,0,0,0,0
BRISTOL,0,0,0,0,0,0,0,0
NORTHAMPTON,0,25000,0,0,0,0,0,0
C1,50000,0,0,0,0,0,0,0
C2,0,0,0,10000,0,0,0,0
C3,0,0,0,0,0,40000,0,0
C4,0,0,0,10000,0,0,0,25000


In [91]:
print('Depots to remaing active')
for d in active_d:
    if active_d[d].x > 0:
        print(d)

Depots to remaing active
BIRMINGHAM
EXERTER
NORTHAMPTON


In [92]:
active_d

{'NEWCASTLE': <gurobi.Var active_d[NEWCASTLE] (value -0.0)>,
 'BIRMINGHAM': <gurobi.Var active_d[BIRMINGHAM] (value 1.0)>,
 'LONDON': <gurobi.Var active_d[LONDON] (value 0.0)>,
 'EXERTER': <gurobi.Var active_d[EXERTER] (value 1.0)>,
 'BRISTOL': <gurobi.Var active_d[BRISTOL] (value -0.0)>,
 'NORTHAMPTON': <gurobi.Var active_d[NORTHAMPTON] (value 1.0)>}